In [ ]:
import json
from pathlib import Path

import numpy as np
import tiktoken

In [ ]:
EVAL_DIR = Path.cwd() / "output_eval_v3"
GPT_4_TURBO_INPUT_COST_PER_1K = 0.01
GPT_4_TURBO_OUTPUT_COST_PER_1K = 0.03
TOKENIZER_NAME = "cl100k_base"

In [ ]:
def find_json_strings(json_object):
    strings_list = []

    def process_object(obj):
        if isinstance(obj, str):
            strings_list.append(obj)
        elif isinstance(obj, list):
            for item in obj:
                process_object(item)
        elif isinstance(obj, dict):
            for value in obj.values():
                process_object(value)

    process_object(json_object)
    return strings_list

In [ ]:
num_extractive_sentences = 0
num_generated_sentences = 0

for dir in EVAL_DIR.iterdir():
    discharge_summary_json = json.loads((dir / "discharge_summary.json").read_text())

    raw_message_text = (dir / "raw_messages.txt").read_text()
    raw_message_sections = raw_message_text.split("\n" + "*" * 80 + "\n")
    physician_notes_text_lowercase = raw_message_sections[3].lower()

    json_strings = find_json_strings(discharge_summary_json)
    json_sentences_lowercase = [
        sentence.lower()
        for item in json_strings
        for sentence in item.split(". ")
        if sentence != ""
    ]

    num_extractive_sentences += sum(
        1
        for sentence_lowercase in json_sentences_lowercase
        if sentence_lowercase in physician_notes_text_lowercase
    )
    num_generated_sentences += len(json_sentences_lowercase)

num_extractive_sentences, num_generated_sentences, num_extractive_sentences / num_generated_sentences

In [ ]:
json_strings

In [ ]:
discharge_summary_json

In [ ]:
tokenizer = tiktoken.get_encoding(TOKENIZER_NAME)

In [ ]:
time_taken = []
input_token_lengths = []
output_token_lengths = []

for dir in EVAL_DIR.iterdir():
    raw_message_text = (dir / "raw_messages.txt").read_text()
    raw_message_sections = raw_message_text.split("\n" + "*" * 80 + "\n")

    input_token_lengths.append(
        sum(len(tokenizer.encode(message)) for message in raw_message_sections[:4])
    )
    output_token_lengths.append(len(tokenizer.encode(raw_message_sections[4])))

    time_taken.append(float(raw_message_sections[-2].split(": ")[1]))

time_taken_np = np.array(time_taken)
input_token_lengths_np = np.array(input_token_lengths)
output_token_lengths_np = np.array(output_token_lengths)

In [ ]:
print(
    time_taken_np.mean(), time_taken_np.std(), time_taken_np.min(), time_taken_np.max()
)

In [ ]:
print(
    input_token_lengths_np.mean(),
    input_token_lengths_np.std(),
    input_token_lengths_np.min(),
    input_token_lengths_np.max(),
)

In [ ]:
print(
    output_token_lengths_np.mean(),
    output_token_lengths_np.std(),
    output_token_lengths_np.min(),
    output_token_lengths_np.max(),
)

In [ ]:
cost = (input_token_lengths_np / 1000 * GPT_4_TURBO_INPUT_COST_PER_1K) + (
    output_token_lengths_np / 1000 * GPT_4_TURBO_OUTPUT_COST_PER_1K
)
print(cost.mean(), cost.std(), cost.min(), cost.max())